# Module 1.3: 直方圖操作 (Histogram Operations)

## 學習目標

本 notebook 將帶你實作影像直方圖相關的操作：

1. **計算直方圖**：從零實作，不使用 np.histogram
2. **直方圖均衡 (Histogram Equalization)**：增強影像對比度
3. **直方圖匹配 (Histogram Matching)**：將影像調整為指定分布

## 前置要求
- 完成 01_convolution.ipynb
- 完成 02_filters.ipynb

## 參考資料
- Gonzalez & Woods, Digital Image Processing, Ch. 3.3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
sys.path.append('../../')

# 載入影像工具
try:
    from utils.image_io import load_grayscale, save_image
except:
    def load_grayscale(path):
        return np.array(Image.open(path).convert('L'), dtype=np.float64)
    def save_image(arr, path):
        Image.fromarray(arr.astype(np.uint8)).save(path)

print("環境設定完成！")

## 建立測試影像

我們建立一些具有不同對比度特性的測試影像。

In [ ]:
def create_low_contrast_image(size=200):
    """
    建立一個低對比度的測試影像。
    所有像素值集中在中間範圍（100-150）。
    """
    np.random.seed(42)
    
    # 基底是灰色
    image = np.ones((size, size), dtype=np.float64) * 125
    
    # 加入一些物件，但對比度很低
    # 左上方塊
    image[30:70, 30:70] = 140
    
    # 右上圓形
    y, x = np.ogrid[:size, :size]
    center_y, center_x = 50, 150
    radius = 25
    mask = (x - center_x)**2 + (y - center_y)**2 <= radius**2
    image[mask] = 110
    
    # 下方漸層
    gradient = np.linspace(115, 135, size)
    image[120:180, 30:170] = gradient[30:170]
    
    # 加入微弱雜訊
    noise = np.random.randn(size, size) * 5
    image = image + noise
    
    return np.clip(image, 0, 255).astype(np.uint8)


def create_dark_image(size=200):
    """
    建立一個整體偏暗的影像。
    像素值集中在 0-80 範圍。
    """
    np.random.seed(42)
    image = np.zeros((size, size), dtype=np.float64)
    
    # 暗的背景
    image[:] = 30
    
    # 物件（也是暗的）
    image[30:70, 30:70] = 60
    image[30:70, 130:170] = 50
    image[130:170, 30:70] = 70
    image[130:170, 130:170] = 40
    
    noise = np.random.randn(size, size) * 5
    image = image + noise
    
    return np.clip(image, 0, 255).astype(np.uint8)


def create_bimodal_image(size=200):
    """
    建立一個雙峰分布的影像。
    大部分像素集中在兩個極端。
    """
    np.random.seed(42)
    image = np.zeros((size, size), dtype=np.float64)
    
    # 棋盤格模式
    block_size = 40
    for i in range(size // block_size):
        for j in range(size // block_size):
            if (i + j) % 2 == 0:
                image[i*block_size:(i+1)*block_size, 
                      j*block_size:(j+1)*block_size] = 50
            else:
                image[i*block_size:(i+1)*block_size, 
                      j*block_size:(j+1)*block_size] = 200
    
    noise = np.random.randn(size, size) * 10
    image = image + noise
    
    return np.clip(image, 0, 255).astype(np.uint8)


# 建立測試影像
low_contrast = create_low_contrast_image()
dark_image = create_dark_image()
bimodal_image = create_bimodal_image()

# 顯示
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(low_contrast, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Low Contrast Image')

axes[1].imshow(dark_image, cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Dark Image')

axes[2].imshow(bimodal_image, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Bimodal Image')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

---

# Part 1: 計算直方圖

## 概念說明

**直方圖 (Histogram)** 是影像中每個灰階值的像素數量統計。

### 定義：

對於灰階影像（0-255），直方圖 $h$ 定義為：

$$
h(k) = \text{number of pixels with intensity } k, \quad k = 0, 1, ..., 255
$$

### 正規化直方圖（機率密度）：

$$
p(k) = \frac{h(k)}{N}
$$

其中 $N$ 是影像的總像素數。

### 直方圖的用途：
1. 分析影像的亮度分布
2. 判斷曝光是否正確
3. 作為影像增強的基礎
4. 用於影像分割（如 Otsu's method）

In [ ]:
def compute_histogram(image, bins=256):
    """
    計算影像的直方圖（不使用 np.histogram）。
    
    Parameters
    ----------
    image : np.ndarray
        輸入影像，uint8 格式 (0-255)
    bins : int
        直方圖的 bin 數量（預設 256）
    
    Returns
    -------
    hist : np.ndarray, shape (bins,)
        直方圖（每個灰階值的像素數量）
    
    實作方式：
    1. 初始化一個長度為 bins 的零陣列
    2. 遍歷影像中的每個像素
    3. 將對應的 bin 計數加 1
    """
    # 確保影像是整數型態
    image = image.astype(np.int32)
    
    # 初始化直方圖
    hist = np.zeros(bins, dtype=np.int64)
    
    # 方法 1：使用 for 迴圈（慢但直觀）
    # for pixel in image.flatten():
    #     if 0 <= pixel < bins:
    #         hist[pixel] += 1
    
    # 方法 2：向量化（快）
    # 扁平化影像
    flat = image.flatten()
    
    # 確保值在有效範圍內
    flat = np.clip(flat, 0, bins - 1)
    
    # 使用 np.bincount 計數（這是純 numpy 操作，不是 np.histogram）
    counts = np.bincount(flat, minlength=bins)
    hist = counts[:bins]
    
    return hist


def compute_histogram_naive(image, bins=256):
    """
    使用 for 迴圈計算直方圖（純教學用）。
    這個版本展示最基本的概念，但速度較慢。
    """
    hist = np.zeros(bins, dtype=np.int64)
    
    # 雙重迴圈遍歷每個像素
    H, W = image.shape
    for i in range(H):
        for j in range(W):
            pixel = int(image[i, j])
            if 0 <= pixel < bins:
                hist[pixel] += 1
    
    return hist

# 測試
hist = compute_histogram(low_contrast)

print(f"直方圖長度：{len(hist)}")
print(f"總像素數：{hist.sum()}")
print(f"影像大小：{low_contrast.shape[0] * low_contrast.shape[1]}")
print(f"驗證：{hist.sum() == low_contrast.shape[0] * low_contrast.shape[1]}")

In [ ]:
def plot_histogram(image, title="", ax=None):
    """
    繪製影像及其直方圖。
    """
    hist = compute_histogram(image)
    
    if ax is None:
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        ax_img, ax_hist = axes
    else:
        ax_img, ax_hist = ax
    
    # 顯示影像
    ax_img.imshow(image, cmap='gray', vmin=0, vmax=255)
    ax_img.set_title(f'{title} Image')
    ax_img.axis('off')
    
    # 繪製直方圖
    ax_hist.bar(range(256), hist, width=1, color='gray', edgecolor='none')
    ax_hist.set_xlim(0, 255)
    ax_hist.set_xlabel('Pixel Intensity')
    ax_hist.set_ylabel('Count')
    ax_hist.set_title(f'{title} Histogram')
    
    return hist

# 繪製三個測試影像的直方圖
fig, axes = plt.subplots(3, 2, figsize=(12, 12))

plot_histogram(low_contrast, "Low Contrast", ax=axes[0])
plot_histogram(dark_image, "Dark", ax=axes[1])
plot_histogram(bimodal_image, "Bimodal", ax=axes[2])

plt.tight_layout()
plt.show()

print("觀察：")
print("1. Low Contrast：像素集中在中間（100-150），分布很窄")
print("2. Dark：像素集中在左邊（0-80），整體偏暗")
print("3. Bimodal：有兩個峰，分別在 50 和 200 附近")

In [ ]:
def compute_cdf(hist):
    """
    計算累積分布函數 (Cumulative Distribution Function)。
    
    Parameters
    ----------
    hist : np.ndarray
        直方圖
    
    Returns
    -------
    cdf : np.ndarray
        累積分布函數
    
    數學定義：
    CDF(k) = sum(hist[0:k+1]) / sum(hist)
           = P(pixel <= k)
    """
    # 正規化直方圖為機率分布
    pdf = hist.astype(np.float64) / hist.sum()
    
    # 計算累積和
    cdf = np.cumsum(pdf)
    
    return cdf

# 繪製 CDF
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

images = [low_contrast, dark_image, bimodal_image]
titles = ['Low Contrast', 'Dark', 'Bimodal']

for ax, image, title in zip(axes, images, titles):
    hist = compute_histogram(image)
    cdf = compute_cdf(hist)
    
    ax.plot(range(256), cdf, 'b-', linewidth=2)
    ax.set_xlim(0, 255)
    ax.set_ylim(0, 1)
    ax.set_xlabel('Pixel Intensity')
    ax.set_ylabel('Cumulative Probability')
    ax.set_title(f'{title} CDF')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("CDF 觀察：")
print("1. 理想的均勻分布：CDF 是一條從 (0,0) 到 (255,1) 的直線")
print("2. Low Contrast：CDF 在中間區域陡峭上升")
print("3. Dark：CDF 在左邊就快速達到 1")
print("4. Bimodal：CDF 有兩段快速上升（對應兩個峰）")

---

# Part 2: 直方圖均衡 (Histogram Equalization)

## 概念說明

**直方圖均衡** 是一種自動增強影像對比度的技術：

- 目標：讓直方圖盡可能均勻分布（flat histogram）
- 效果：增強影像對比度，讓細節更明顯

### 數學原理

對於輸入影像的每個像素值 $r$，輸出像素值 $s$ 定義為：

$$
s = T(r) = (L-1) \cdot CDF(r)
$$

其中：
- $L$ 是灰階數（通常是 256）
- $CDF(r)$ 是累積分布函數在 $r$ 處的值

### 直覺解釋：

1. CDF 將像素從「原本的灰階值」映射到「該灰階值以下的像素比例」
2. 乘以 $(L-1)$ 將比例轉換回灰階值
3. 結果：常見的灰階值會被「拉開」，罕見的會被「壓縮」

In [ ]:
def histogram_equalization(image):
    """
    對灰階影像進行直方圖均衡。
    
    Parameters
    ----------
    image : np.ndarray
        輸入灰階影像 (uint8, 0-255)
    
    Returns
    -------
    equalized : np.ndarray
        直方圖均衡後的影像 (uint8, 0-255)
    
    實作步驟：
    1. 計算直方圖
    2. 計算 CDF
    3. 建立映射表：new_value[k] = round((L-1) * CDF[k])
    4. 對每個像素套用映射
    """
    # Step 1: 計算直方圖
    hist = compute_histogram(image, bins=256)
    
    # Step 2: 計算 CDF
    cdf = compute_cdf(hist)
    
    # Step 3: 建立映射表
    # 將 CDF (0~1) 映射到 0~255
    L = 256  # 灰階數
    mapping = np.round((L - 1) * cdf).astype(np.uint8)
    
    # Step 4: 套用映射
    # 使用 mapping 作為 lookup table
    equalized = mapping[image.astype(np.int32)]
    
    return equalized.astype(np.uint8)

# 測試
low_eq = histogram_equalization(low_contrast)
dark_eq = histogram_equalization(dark_image)
bimodal_eq = histogram_equalization(bimodal_image)

print("直方圖均衡完成！")

In [ ]:
def compare_before_after(original, equalized, title):
    """
    比較直方圖均衡前後的差異。
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Original image and histogram
    axes[0, 0].imshow(original, cmap='gray', vmin=0, vmax=255)
    axes[0, 0].set_title(f'{title} - Original')
    axes[0, 0].axis('off')
    
    hist_orig = compute_histogram(original)
    axes[0, 1].bar(range(256), hist_orig, width=1, color='gray', edgecolor='none')
    axes[0, 1].set_xlim(0, 255)
    axes[0, 1].set_title('Original Histogram')
    axes[0, 1].set_xlabel('Pixel Intensity')
    axes[0, 1].set_ylabel('Count')
    
    # Equalized image and histogram
    axes[1, 0].imshow(equalized, cmap='gray', vmin=0, vmax=255)
    axes[1, 0].set_title(f'{title} - Equalized')
    axes[1, 0].axis('off')
    
    hist_eq = compute_histogram(equalized)
    axes[1, 1].bar(range(256), hist_eq, width=1, color='blue', edgecolor='none', alpha=0.7)
    axes[1, 1].set_xlim(0, 255)
    axes[1, 1].set_title('Equalized Histogram')
    axes[1, 1].set_xlabel('Pixel Intensity')
    axes[1, 1].set_ylabel('Count')
    
    plt.tight_layout()
    plt.show()

# 比較三個影像
compare_before_after(low_contrast, low_eq, "Low Contrast")

In [ ]:
compare_before_after(dark_image, dark_eq, "Dark")

In [ ]:
compare_before_after(bimodal_image, bimodal_eq, "Bimodal")

In [ ]:
# CDF 比較
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

pairs = [
    (low_contrast, low_eq, "Low Contrast"),
    (dark_image, dark_eq, "Dark"),
    (bimodal_image, bimodal_eq, "Bimodal")
]

for ax, (orig, eq, title) in zip(axes, pairs):
    cdf_orig = compute_cdf(compute_histogram(orig))
    cdf_eq = compute_cdf(compute_histogram(eq))
    ideal = np.linspace(0, 1, 256)  # 理想的均勻分布
    
    ax.plot(range(256), cdf_orig, 'r-', label='Original', linewidth=2)
    ax.plot(range(256), cdf_eq, 'b-', label='Equalized', linewidth=2)
    ax.plot(range(256), ideal, 'g--', label='Ideal Uniform', linewidth=1, alpha=0.7)
    
    ax.set_xlim(0, 255)
    ax.set_ylim(0, 1)
    ax.set_xlabel('Pixel Intensity')
    ax.set_ylabel('Cumulative Probability')
    ax.set_title(f'{title} CDF Comparison')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("觀察：")
print("1. 均衡後的 CDF（藍線）更接近理想的直線（綠虛線）")
print("2. 但由於離散性，均衡後的 CDF 仍會有階梯狀")
print("3. 這就是為什麼均衡後的直方圖不是完全 flat，而是較均勻")

---

# Part 3: 深入理解映射過程

讓我們詳細觀察直方圖均衡的映射過程。

In [ ]:
def visualize_mapping(image, title=""):
    """
    視覺化直方圖均衡的映射過程。
    """
    # 計算直方圖和 CDF
    hist = compute_histogram(image, bins=256)
    cdf = compute_cdf(hist)
    
    # 建立映射表
    mapping = np.round(255 * cdf).astype(np.uint8)
    
    # 繪製映射函數
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 1. 原始直方圖
    axes[0].bar(range(256), hist, width=1, color='gray', edgecolor='none')
    axes[0].set_xlim(0, 255)
    axes[0].set_xlabel('Input Intensity')
    axes[0].set_ylabel('Count')
    axes[0].set_title(f'{title} Original Histogram')
    
    # 2. 映射函數 T(r)
    axes[1].plot(range(256), mapping, 'b-', linewidth=2)
    axes[1].plot([0, 255], [0, 255], 'r--', linewidth=1, label='Identity (no change)')
    axes[1].set_xlim(0, 255)
    axes[1].set_ylim(0, 255)
    axes[1].set_xlabel('Input Intensity (r)')
    axes[1].set_ylabel('Output Intensity (s)')
    axes[1].set_title('Mapping Function T(r)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    axes[1].set_aspect('equal')
    
    # 3. CDF
    axes[2].plot(range(256), cdf, 'g-', linewidth=2)
    axes[2].set_xlim(0, 255)
    axes[2].set_ylim(0, 1)
    axes[2].set_xlabel('Intensity')
    axes[2].set_ylabel('CDF')
    axes[2].set_title('CDF (normalized mapping)')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return mapping

# 視覺化各種影像的映射
print("=== Low Contrast Image ===")
mapping_lc = visualize_mapping(low_contrast, "Low Contrast")

print("\n觀察：")
print("1. 原本像素集中在 100-150")
print("2. 映射函數在這個區域斜率很大（陡峭）")
print("3. 這會把密集的像素值「拉開」，增加對比度")

In [ ]:
print("=== Dark Image ===")
mapping_dark = visualize_mapping(dark_image, "Dark")

print("\n觀察：")
print("1. 原本像素集中在 0-80（偏暗）")
print("2. 映射函數把暗像素映射到更寬的範圍")
print("3. 影像整體會變亮")

In [ ]:
print("=== Bimodal Image ===")
mapping_bi = visualize_mapping(bimodal_image, "Bimodal")

print("\n觀察：")
print("1. 有兩個峰（50 和 200）")
print("2. 映射函數有兩段陡峭區域")
print("3. 兩個峰之間的「空隙」會被壓縮")

---

# Part 4: CLAHE（進階主題）

## Contrast Limited Adaptive Histogram Equalization

標準直方圖均衡的問題：
1. 全域操作，可能過度增強某些區域
2. 可能放大雜訊

**CLAHE** 的解決方案：
1. 將影像分成小區塊（tiles）
2. 對每個區塊分別做直方圖均衡
3. **限制對比度**：限制直方圖的高度，防止過度增強
4. 使用雙線性插值融合區塊邊界

In [ ]:
def clip_histogram(hist, clip_limit):
    """
    限制直方圖的高度（CLAHE 的核心概念）。
    
    超過 clip_limit 的部分會被重新分配到其他 bins。
    
    Parameters
    ----------
    hist : np.ndarray
        原始直方圖
    clip_limit : float
        每個 bin 的最大值（相對於平均值的倍數）
    
    Returns
    -------
    clipped_hist : np.ndarray
        限制後的直方圖
    """
    hist = hist.astype(np.float64)
    
    # 計算實際的 clip 值
    avg = hist.mean()
    limit = clip_limit * avg
    
    # 計算超過限制的總量
    excess = np.sum(np.maximum(hist - limit, 0))
    
    # 限制直方圖
    clipped = np.minimum(hist, limit)
    
    # 將超過的部分平均分配
    redistribution = excess / len(hist)
    clipped = clipped + redistribution
    
    return clipped


def clahe_simple(image, clip_limit=2.0, grid_size=8):
    """
    簡化版的 CLAHE 實作。
    
    注意：這是教學用的簡化版，沒有實作雙線性插值。
    完整的 CLAHE 還需要在區塊邊界做平滑處理。
    
    Parameters
    ----------
    image : np.ndarray
        輸入影像
    clip_limit : float
        對比度限制（相對於平均值的倍數）
    grid_size : int
        區塊數量（每個方向）
    
    Returns
    -------
    result : np.ndarray
        處理後的影像
    """
    H, W = image.shape
    result = np.zeros_like(image, dtype=np.float64)
    
    # 計算每個區塊的大小
    tile_h = H // grid_size
    tile_w = W // grid_size
    
    for i in range(grid_size):
        for j in range(grid_size):
            # 取出區塊
            y1 = i * tile_h
            y2 = (i + 1) * tile_h if i < grid_size - 1 else H
            x1 = j * tile_w
            x2 = (j + 1) * tile_w if j < grid_size - 1 else W
            
            tile = image[y1:y2, x1:x2]
            
            # 計算並限制直方圖
            hist = compute_histogram(tile)
            clipped_hist = clip_histogram(hist, clip_limit)
            
            # 計算 CDF 並建立映射
            cdf = np.cumsum(clipped_hist) / clipped_hist.sum()
            mapping = np.round(255 * cdf).astype(np.uint8)
            
            # 套用映射
            result[y1:y2, x1:x2] = mapping[tile.astype(np.int32)]
    
    return result.astype(np.uint8)

# 測試 CLAHE
clahe_result = clahe_simple(low_contrast, clip_limit=2.0, grid_size=4)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(low_contrast, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(low_eq, cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Standard Histogram Equalization')

axes[2].imshow(clahe_result, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('CLAHE (simplified)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("注意：這是簡化版 CLAHE，區塊邊界可能有明顯接縫")
print("完整版需要雙線性插值來平滑邊界")

---

# Part 5: 練習題

## 練習 1: 計算直方圖統計量

In [ ]:
def histogram_statistics(image):
    """
    從直方圖計算影像的統計量。
    
    Parameters
    ----------
    image : np.ndarray
        輸入影像
    
    Returns
    -------
    stats : dict
        包含以下統計量：
        - mean: 平均灰階值
        - std: 標準差
        - median: 中位數
        - mode: 眾數（最常出現的灰階值）
        - dynamic_range: 動態範圍（最大值 - 最小值）
    
    提示：
    - 使用直方圖計算這些統計量
    - mean = sum(k * p(k)) for k in 0..255
    - variance = sum((k - mean)^2 * p(k))
    """
    hist = compute_histogram(image)
    pdf = hist / hist.sum()  # 正規化為機率分布
    
    # 灰階值
    k = np.arange(256)
    
    # 計算平均值
    mean = np.sum(k * pdf)
    
    # 計算標準差
    variance = np.sum((k - mean)**2 * pdf)
    std = np.sqrt(variance)
    
    # 計算中位數（CDF 達到 0.5 的位置）
    cdf = np.cumsum(pdf)
    median = np.searchsorted(cdf, 0.5)
    
    # 計算眾數（直方圖最高點）
    mode = np.argmax(hist)
    
    # 計算動態範圍
    nonzero = np.where(hist > 0)[0]
    if len(nonzero) > 0:
        dynamic_range = nonzero[-1] - nonzero[0]
    else:
        dynamic_range = 0
    
    return {
        'mean': mean,
        'std': std,
        'median': median,
        'mode': mode,
        'dynamic_range': dynamic_range
    }

# 測試
print("Low Contrast Image Statistics:")
stats_lc = histogram_statistics(low_contrast)
for key, val in stats_lc.items():
    print(f"  {key}: {val:.2f}")

print("\nAfter Equalization:")
stats_lc_eq = histogram_statistics(low_eq)
for key, val in stats_lc_eq.items():
    print(f"  {key}: {val:.2f}")

print("\n觀察：")
print(f"1. 動態範圍從 {stats_lc['dynamic_range']:.0f} 增加到 {stats_lc_eq['dynamic_range']:.0f}")
print(f"2. 標準差從 {stats_lc['std']:.1f} 增加到 {stats_lc_eq['std']:.1f}（分布更寬）")

## 練習 2: 直方圖匹配（Histogram Matching / Specification）

將一張影像的直方圖調整成另一張影像的直方圖分布。

In [ ]:
def histogram_matching(source, reference):
    """
    將 source 影像的直方圖匹配到 reference 影像。
    
    Parameters
    ----------
    source : np.ndarray
        需要調整的影像
    reference : np.ndarray
        目標分布的參考影像
    
    Returns
    -------
    matched : np.ndarray
        匹配後的影像
    
    實作步驟：
    1. 計算 source 的 CDF: G(r)
    2. 計算 reference 的 CDF: H(z)
    3. 對於每個 source 灰階值 r，找到滿足 H(z) ≈ G(r) 的 z
    4. 建立映射 r -> z
    """
    # Step 1: 計算 source 的 CDF
    hist_src = compute_histogram(source)
    cdf_src = compute_cdf(hist_src)
    
    # Step 2: 計算 reference 的 CDF
    hist_ref = compute_histogram(reference)
    cdf_ref = compute_cdf(hist_ref)
    
    # Step 3: 建立映射
    # 對於每個 source 灰階值，找到 reference CDF 中最接近的值
    mapping = np.zeros(256, dtype=np.uint8)
    
    for r in range(256):
        # 找到 cdf_ref 中最接近 cdf_src[r] 的位置
        target = cdf_src[r]
        # 使用 searchsorted 找到插入點
        z = np.searchsorted(cdf_ref, target)
        z = min(z, 255)  # 確保不超過範圍
        mapping[r] = z
    
    # Step 4: 套用映射
    matched = mapping[source.astype(np.int32)]
    
    return matched.astype(np.uint8)

# 測試：將 dark image 的分布匹配到 low contrast image
matched = histogram_matching(dark_image, low_contrast)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Row 1: 影像
axes[0, 0].imshow(dark_image, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Source (Dark)')

axes[0, 1].imshow(low_contrast, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title('Reference (Low Contrast)')

axes[0, 2].imshow(matched, cmap='gray', vmin=0, vmax=255)
axes[0, 2].set_title('Matched')

# Row 2: 直方圖
axes[1, 0].bar(range(256), compute_histogram(dark_image), width=1, color='gray')
axes[1, 0].set_xlim(0, 255)
axes[1, 0].set_title('Source Histogram')

axes[1, 1].bar(range(256), compute_histogram(low_contrast), width=1, color='gray')
axes[1, 1].set_xlim(0, 255)
axes[1, 1].set_title('Reference Histogram')

axes[1, 2].bar(range(256), compute_histogram(matched), width=1, color='blue', alpha=0.7)
axes[1, 2].set_xlim(0, 255)
axes[1, 2].set_title('Matched Histogram')

for ax in axes[0]:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("觀察：")
print("1. Matched 影像的直方圖分布接近 Reference")
print("2. 但內容（形狀、紋理）來自 Source")

## 練習 3: 完整的直方圖處理類別

In [ ]:
class HistogramProcessor:
    """
    直方圖處理工具類別。
    
    整合所有直方圖相關的操作。
    """
    
    @staticmethod
    def compute_histogram(image, bins=256):
        """計算直方圖"""
        flat = image.flatten().astype(np.int32)
        flat = np.clip(flat, 0, bins - 1)
        return np.bincount(flat, minlength=bins)[:bins]
    
    @staticmethod
    def compute_cdf(hist):
        """計算 CDF"""
        pdf = hist.astype(np.float64) / hist.sum()
        return np.cumsum(pdf)
    
    @classmethod
    def equalize(cls, image):
        """直方圖均衡"""
        hist = cls.compute_histogram(image)
        cdf = cls.compute_cdf(hist)
        mapping = np.round(255 * cdf).astype(np.uint8)
        return mapping[image.astype(np.int32)].astype(np.uint8)
    
    @classmethod
    def match(cls, source, reference):
        """直方圖匹配"""
        cdf_src = cls.compute_cdf(cls.compute_histogram(source))
        cdf_ref = cls.compute_cdf(cls.compute_histogram(reference))
        
        mapping = np.zeros(256, dtype=np.uint8)
        for r in range(256):
            z = np.searchsorted(cdf_ref, cdf_src[r])
            mapping[r] = min(z, 255)
        
        return mapping[source.astype(np.int32)].astype(np.uint8)
    
    @classmethod
    def statistics(cls, image):
        """計算統計量"""
        hist = cls.compute_histogram(image)
        pdf = hist / hist.sum()
        k = np.arange(256)
        
        mean = np.sum(k * pdf)
        std = np.sqrt(np.sum((k - mean)**2 * pdf))
        cdf = np.cumsum(pdf)
        median = np.searchsorted(cdf, 0.5)
        mode = np.argmax(hist)
        
        nonzero = np.where(hist > 0)[0]
        dynamic_range = nonzero[-1] - nonzero[0] if len(nonzero) > 0 else 0
        
        return {
            'mean': mean,
            'std': std,
            'median': median,
            'mode': mode,
            'dynamic_range': dynamic_range
        }

# 測試類別
print("=== HistogramProcessor 測試 ===")

hp = HistogramProcessor

# 測試各種功能
hist = hp.compute_histogram(low_contrast)
print(f"直方圖總和：{hist.sum()} (應等於 {low_contrast.size})")

eq = hp.equalize(low_contrast)
print(f"均衡後影像形狀：{eq.shape}")

matched = hp.match(dark_image, low_contrast)
print(f"匹配後影像形狀：{matched.shape}")

stats = hp.statistics(low_contrast)
print(f"統計量：{stats}")

print("\n所有測試通過！")

---

# 總結

本 notebook 實作了以下直方圖操作：

| 功能 | 用途 | 關鍵概念 |
|------|------|----------|
| compute_histogram | 計算灰階分布 | 統計每個灰階值的像素數 |
| compute_cdf | 計算累積分布 | 用於均衡和匹配 |
| histogram_equalization | 增強對比度 | 將 CDF 映射為線性 |
| histogram_matching | 風格轉換 | 將分布匹配到參考影像 |
| CLAHE | 局部對比度增強 | 分區塊 + 對比度限制 |

## 關鍵概念

1. **直方圖** 描述影像的灰階分布
2. **CDF** 是直方圖均衡的核心工具
3. **直方圖均衡** 自動增強對比度
4. **直方圖匹配** 可用於風格轉換

## 下一步

- 在 Module 2 中，我們會實作 **Otsu's Method**，使用直方圖自動選擇最佳閾值進行影像分割

In [ ]:
# 最終驗證
print("=== 函數驗證 ===")

test_funcs = [
    ("compute_histogram(image)", lambda: compute_histogram(low_contrast)),
    ("compute_histogram_naive(image)", lambda: compute_histogram_naive(low_contrast[:50, :50])),
    ("compute_cdf(hist)", lambda: compute_cdf(compute_histogram(low_contrast))),
    ("histogram_equalization(image)", lambda: histogram_equalization(low_contrast)),
    ("histogram_matching(src, ref)", lambda: histogram_matching(dark_image, low_contrast)),
    ("histogram_statistics(image)", lambda: histogram_statistics(low_contrast)),
    ("clip_histogram(hist, limit)", lambda: clip_histogram(compute_histogram(low_contrast), 2.0)),
    ("clahe_simple(image)", lambda: clahe_simple(low_contrast)),
    ("HistogramProcessor.equalize(image)", lambda: HistogramProcessor.equalize(low_contrast)),
    ("HistogramProcessor.match(src, ref)", lambda: HistogramProcessor.match(dark_image, low_contrast)),
]

for name, func in test_funcs:
    try:
        result = func()
        print(f"✓ {name}")
    except Exception as e:
        print(f"✗ {name}: {e}")

print("\n所有函數驗證完成！")